In [1]:
import sys, os
sys.path.append(os.getcwd())

In [2]:
from models import EnsembleRegressor, Objective
from kennard_stone import train_test_split, KFold

import pandas as pd
from sklearn.metrics import mean_squared_error
import optuna
from lightgbm import LGBMRegressor

Using TensorFlow backend.


In [3]:
SEED = 334
kf = KFold(n_splits = 5)

In [4]:
from sklearn.datasets import load_boston
data = load_boston()
X = pd.DataFrame(data.data, columns = data.feature_names)
y = pd.Series(data.target, name = 'PRICE')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [6]:
# rf = RandomForestRegressor(random_state = SEED, n_jobs = -1)
lgbt = LGBMRegressor(random_state = SEED, n_jobs = -1)

In [7]:
objective = Objective(lgbt, X_train, y_train, scoring = 'neg_root_mean_squared_error', cv = kf)
sampler = optuna.samplers.TPESampler(seed = SEED)
study = optuna.create_study(sampler = sampler, direction = 'maximize')
study.optimize(objective, n_trials = 100, n_jobs = -1)

[I 2021-01-31 21:16:31,709] A new study created in memory with name: no-name-d428c36a-3403-48e4-960b-d70957280579
[I 2021-01-31 21:16:38,664] Trial 6 finished with value: -3.631742865049717 and parameters: {'n_estimators': 67, 'min_child_weight': 0.1344097977727346, 'colsample_bytree': 0.6834391857605651, 'subsample': 0.9357626252440614, 'num_leaves': 26}. Best is trial 6 with value: -3.631742865049717.
[I 2021-01-31 21:16:39,319] Trial 5 finished with value: -3.588805151049802 and parameters: {'n_estimators': 80, 'min_child_weight': 0.009796111638639676, 'colsample_bytree': 0.7880101867793383, 'subsample': 0.8220073969583326, 'num_leaves': 375}. Best is trial 5 with value: -3.588805151049802.
[I 2021-01-31 21:16:44,690] Trial 0 finished with value: -3.5375174187241045 and parameters: {'n_estimators': 164, 'min_child_weight': 0.002992655467502613, 'colsample_bytree': 0.909170757405618, 'subsample': 0.8116783246663085, 'num_leaves': 399}. Best is trial 0 with value: -3.5375174187241045.

In [8]:
best_estimator = objective.model(**objective.fixed_params_, **study.best_params).fit(X_train, y_train)

In [9]:
mean_squared_error(besNNt_estimator.predict(X_test), y_test, squared = False)

2.279936228259161

同じ条件にするために```boruta```を```False```に．

In [10]:
er = EnsembleRegressor([lgbt], random_state = SEED, n_jobs = -1, boruta = False, scoring = 'neg_root_mean_squared_error', verbose = 0, cv = kf)

In [11]:
er.fit(X_train, y_train)

EnsembleRegressor(boruta=False, cv=KFold(n_splits=5),
                  estimators=[LGBMRegressor(random_state=334)],
                  random_state=334, scoring='neg_root_mean_squared_error',
                  verbose=0)

In [12]:
mean_squared_error(er.predict(X_test), y_test, squared = False)

2.189592488626531

In [13]:
for estimators in er.results_.estimators:
    print(mean_squared_error(estimators[0].predict(X_test), y_test, squared = False))

2.2958052728000737
2.3917277246043858
2.471371097501595
2.5692955116163643
2.66576030132261
